In [10]:
# Dependências
import gensim
import pyemd
from gensim.models import Word2Vec
import pandas as pd
import math
import pickle

In [2]:
# Desabilitando warnings 
import warnings
warnings.filterwarnings('ignore')

In [12]:
# Lendo o arquivo base de noticias e extraindo apenas os titles
def limpa_frase(frase):
    palavras = frase.split()
    filtradas = [p.lower() for p in palavras if len(p) >= 3]
    return ' '.join(filtradas)
            

link = 'https://raw.githubusercontent.com/geovanens/myserver/master/results.csv'
results = pd.read_csv(link)
documentos = results['text']
noticias = results['title']

noticias_filtradas = [limpa_frase(n) for n in noticias] # apenas titles
noticias_filtradas

['sociedade foi rubens paiva não facínoras que mataram”',
 'justiça suspende decisão que proibia forças armadas celebrarem golpe 1964',
 'governo bolsonaro prega “negacionismo histórico” sobre ditadura',
 'quando pais gabo perceberam que tinham filho mentiroso',
 'rádios canadenses banem músicas michael jackson programação',
 'rosângela uma das crianças sequestradas por famílias militares ditadura',
 '“lógica usar torturadores ditadura crime foi usada nas milícias”',
 'bolsonaro troca embaixada por escritório jerusalém mas não evita retaliação palestina',
 'lado mais sombrio dickens',
 'amor entre mulheres que sacudiu paraguai',
 'feminista susan sontag contra norman mailer',
 'imagens questionadoras harun farocki',
 'manuscrito ‘cem anos solidão’ que garcía márquez acreditava ter perdido',
 'ricardo olha para tela celular diz: “eu estou falando comigo mesmo”',
 'morte inocente neto lula soltou monstros ódio',
 'tragédia ucrânia',
 'por que papa francisco ordenou abertura dos arquivos 

In [4]:
# Lendo o modelo
model = Word2Vec.load('pt.bin')

### 1. Implemente uma função que recebe uma notícia e retorna os vetores (word embeddings) das palavras do título dessa notícia a partir dos word embeddings pré-treinados com o gensim (30 pts).

In [15]:
def get_vetores(noticia):
    vetores = {}
    for p in noticia.split():
        if p in model:
            vetores[p] = model[p]
    return vetores

# get_vetores(noticias_filtradas[0])

### 2. Implemente uma função que calcula o WMD (Word Mover's Distance) entre duas notícias usando os embeddings das palavras dos respectivos títulos  (35 pts);

In [13]:
def WMD2(noticia1, noticia2):
    wmd = model.wmdistance(noticia1.lower().split(), noticia2.lower().split())
    return wmd

def WMD(noticia1, noticia2):
    vetores1 = get_vetores(noticia1).values()
    vetores2 = get_vetores(noticia2).values()
  
    wmd = 0
    for emb1 in vetores1:
        min_dist = math.inf
        for emb2 in vetores2:
            try:
                min_dist = min(min_dist, math.sqrt(sum((emb1 - emb2) ** 2)))
            except:
                pass
        wmd += min_dist * 1.0/len(vetores1)
 
    return wmd

In [17]:
#  WMD entre duas noticias sem relação
n1 = noticias_filtradas[16]
n2 = noticias_filtradas[0]
print(n1)
print(n2)
WMD(n1, n2)

por que papa francisco ordenou abertura dos arquivos secretos vaticano sobre holocausto judeu?
sociedade foi rubens paiva não facínoras que mataram”


23.213619211129174

#### Notamos um WMD alto para noticias sem relação

In [19]:
#  WMD entre duas noticias com relação
n1 = noticias_filtradas[43]
n2 = noticias_filtradas[84]
print(n1)
print(n2)
WMD(n1, n2)

onde como assistir real madrid barcelona pelo campeonato espanhol
onde como assistir real madrid barcelona pelas semifinais copa rei


8.672602805394636

#### Notamos um WMD baixo para noticias com relação

In [22]:
# Não precisa ser executado pois as distâncias foram previamente salvas no disco
def get_distancias():
    distancias = {}
    for n in noticias_filtradas:
        print(noticias_filtradas.index(n))
        distancias[n] = []
        for n2 in noticias_filtradas:
            if n2 != n:
                distancia = (n2, WMD(n, n2))
                distancias[n].append(distancia)
    return distancias

In [21]:
distancias = pickle.loads(open('distancias.pickle', "rb").read())

# Salvo no disco pois é muito demorado a execução
# O arquivo distancias contem todas as distancias WMD entre todos os titles
# O arquivo é importante para o desenvolvimento da questão 3

### 3. Implemente uma função que possa receber qualquer notícia como entrada e retornar as top-3 notícias mais similares (menos distantes) a ela (35 pts).


In [24]:
def top3(noticia):
    ordenadas = sorted(distancias[noticia], key=lambda x: x[1])
    return ordenadas[:3]

n1 = noticias_filtradas[43]
print(n1)
top3(n1)

onde como assistir real madrid barcelona pelo campeonato espanhol


[('onde como assistir real madrid barcelona pelas semifinais copa rei',
  8.672602805394636),
 ('barcelona volta vencer real madrid encaminha título campeonato espanhol',
  10.29236028502499),
 ('onde como assistir barcelona lyon pela champions league',
  12.00258181497641)]

#### Podemos notar que as top3 noticias relacionadas a n1 acima são totalmente válidas, tratando do mesmo contexto de n1